In [93]:
# email2inventory
# read evergreen email
# this script will fail if scripts/email2inventory/data/evergreen-20200930.txt doesn't exist

import os
import re

def evergreen_read(filename):
    date = ""
    product_flag = False
    position = 0
    items = {}
    f = open(filename)
    lines = f.readlines()
    while not product_flag:
        line = lines.pop(0).strip()
        m = re.match(r"Date: (\w* \d+, \d{4})", line)
        if m:
            date = m.group(1)
        if line == "Product":
            product_flag = True
    line = lines.pop(0).strip()
    assert line == "Quantity"
    line = lines.pop(0).strip()
    assert line == "Price"
    while product_flag:
        m = re.match(r"(.*) - 1 Week", lines.pop(0).strip())
        try:
            itemname = m.group(1)
        except:
            break
        quantity = lines.pop(0).strip()
        price = lines.pop(0).strip()
        items[itemname] = [quantity,price]
    f.close()
    return({'items': items, 'date' : date})
    
print(evergreen_read(os.path.join("email_data/evergreen-20200930.txt")))

{'items': {"Red's Best Hake": ['2', '$32.00']}, 'date': 'September 30, 2020'}


In [166]:
# email2inventory
# read kistler email
# this script will fail if scripts/email2inventory/data/kistler-20200912.txt doesn't exist

import os
import re
import requests
import datetime

client_secret = subprocess.check_output("./sortly_key.sh").strip()
post_url = 'https://api.sortly.co/api/v1/items'

def kistler_read(filename):
    date = ""
    product_flag = False
    position = 0
    items = {}
    f = open(filename)
    lines = f.readlines()
    while not product_flag and lines:
        line = lines.pop(0).strip()
# grabbing the date from the email header
        m = re.match(r"Date: (\w* \d+, \d{4})", line)
        if m:
            date = datetime.datetime.strptime(m.group(1), '%B %d, %Y')
            iso_date = str(date_time_obj).replace('+00:00', 'Z')
# this version still assumes that there is a single item which is a several sets of wines
# in a bundle - need to generalize
        elif line == "Total":
            product_flag = True
        while product_flag:
            m = re.search(r"(\d+)-Bottle Mixed Package\*$", lines.pop(0).strip())
            items = get_package(int(m.group(1)),lines)
            product_flag = False
    f.close()
    return({'items' : items, 'iso_date' : iso_date})

def get_package(bottles,lines):
    inventory = {}
    while bottles > 0:
        line = lines.pop(0).strip()
        m = re.match(r"\•(\d+) btls (\d{4}) (.*)$",line)
        inventory[m.group(3)] = [int(m.group(1)), m.group(2)]
        bottles -= int(m.group(1))
    return inventory
    
inventory = kistler_read(os.path.join("email_data/kistler-20200912.txt"))

for item in inventory['items']:
#    print(item)
#    print(inventory['items'][item])
#    print(inventory['items'][item][0])
# posting to Sortly API
    post_url = 'https://api.sortly.co/api/v1/items'
    data = {'name' : item, 'quantity' : inventory['items'][item][0], 'type' : 'item', 'tags' : [{'name' : 'test'}, {'name' : 'wine'}], 'custom_attribute_values': [{'value': inventory['iso_date'], 'custom_attribute_id': 170931, 'custom_attribute_name': 'Purchase Date'}]}
    response = requests.post(post_url, json=data, headers=headers)

McCrea Vineyard Chardonnay Sonoma Mountain
[2, '2018']
2
Durell Vineyard Chardonnay Sonoma Coast
[2, '2018']
2
Laguna Ridge Vineyard Pinot Noir Russian River Valley
[2, '2018']
2


This uses OAUTH2 to get the two folders from the Sortly API and then open an item and get the image.

Boris figured out how to do it using `urllib.request`, but I think I should be able to do it using `requests`.  Also, I think we're doing some sort of brute force OAUTH and that there is something more complicated but proper - maybe using requests. Something to try to figure out.

I moved the client key to sortly_key.sh as a file that prints the key. I ignored it in GitHub and am not syncing it.

Now trying to generalize for the farmers to you email

In [107]:
# email2inventory
# read farmers to you email
# this script will fail if scripts/email2inventory/data/f2u-20201006.txt doesn't exist

import os
import re
import requests
import datetime
import subprocess
import urllib.request
import json
import io
import subprocess

client_secret = subprocess.check_output("./sortly_key.sh").strip()
working_path = os.getenv("HOME") + '/GitHub/joi.github.io/__scripts/'
post_url = 'https://api.sortly.co/api/v1/items'
product_pattern = 'Item'
pop_fields = ['Item', 'Wt', '$/lb', 'Qty', 'Cost']
sortly_fields = ['name', 'notes', '', 'quantity', 'price']
end_items = ""
# data_file = 'email_data/f2u-20201006.txt'
data_file = 'email_data/f2u-oneitem.txt'


def get_file(filename):
    date = ""
    product_flag = False
    position = 0
    items = {}
    
    print("opening file" + data_file)
    f = open(working_path + filename)
    lines = f.readlines()
    
    while not product_flag and position < len(lines):
        line = lines[position].strip()
        position += 1
# grabbing the date from the email header
        m = re.match(r"Date: (\w* \d+, \d{4})", line)
        if m:
            date_time_obj = datetime.datetime.strptime(m.group(1), '%B %d, %Y')
            iso_date = str(date_time_obj.isoformat()).replace('00:00:00', '00:00:00.000Z')
# if product_pattern matches a line, it means that the part of the email with the item
# list has begun so start reading the products
        elif line == product_pattern:
            product_flag = True
# iterate past each line with a field from the header of the item list
            for field in pop_fields[1:]:
                line = lines[position].strip()
                position += 1
        while product_flag:
# now read in the actual product data
# iterate through each field assuming each new line is another field
            for field_sortly in sortly_fields:
# if it is a field that is not in sortly, ignore
                if field_sortly == '':
                    position +=1
# if the field is the name, create a new dictionary as a value for a dictionary entry with the
# name as the key
                elif field_sortly == 'name':
                    name = lines[position].strip()
                    position += 1
                    if name not in items:
                        items[name] = {}
                    else:
# if there is more than one of the same item name, add a number at the end of the name and increment it
# each time
                        i = 1
                        while name + " " + str(i) in items:
                            i += 1
                        name = name + " " + str(i)
                        items[name] = {}
# if the field is not the name, assign the value of the line to a dictionary entry for dictionary entry
# with the key of the name of the field inside of the entry for the item name
# catch special cases where fields are numbers such as quantity and price
                else:
                    if field_sortly == 'quantity':
                        items[name][field_sortly] = int(lines[position].strip())
                    elif field_sortly == 'price':
                        items[name][field_sortly] = float(lines[position].strip()[1:])
                    else:
                        items[name][field_sortly] = lines[position].strip()
                    position += 1
            line = lines[position].strip()
            if line == end_items:
                product_flag = False
    f.close()
    return items, iso_date

def get_package(bottles,lines):
# for getting bottles inside of a Kistler mixed set of bottles
    inventory = {}
    while bottles > 0:
        line = lines.pop(0).strip()
        m = re.match(r"\•(\d+) btls (\d{4}) (.*)$",line)
        inventory[m.group(3)] = [int(m.group(1)), m.group(2)]
        bottles -= int(m.group(1))
    return inventory

def post_shortly(inventory, date):
# sortly POST API URL
    url = 'https://api.sortly.co/api/v1/items'
# get my secret key
    client_secret = subprocess.check_output("./sortly_key.sh").strip()
    headers = {'Accept': 'application/json','Authorization': client_secret}
    data = {}
    for item in inventory:
        data = {'name' : str(item)}
        for key, value in inventory[item].items():
            data[key] = value
        data['type'] = 'item'
        data['tags'] = [{'name' : 'pending'}]
        data['custom_attribute_values'] = [{'value' : date, 'custom_attribute_id': 170931, 'custom_attribute_name': 'Purchase Date' },{'value': None, 'custom_attribute_id': 171873, 'custom_attribute_name': 'Product URL'}]
        print("posting " + str(item))
# add the item to sortly
        response = requests.post(url, json=data, headers=headers)

inventory, date = get_file(data_file)
post_shortly(inventory, date)


opening fileemail_data/f2u-oneitem.txt
posting Chicken - 4-5.5 lbs Medium
posting Milk - Creamline Organic - Strfrd


In [90]:
# trying to access Sortly API
import urllib.request
import json
import requests
import io
import subprocess
from PIL import Image

client_secret = subprocess.check_output("./sortly_key.sh").strip()

headers = {
  'Accept': 'application/json',
  'Authorization': client_secret
}
response = requests.get('https://api.sortly.co/api/v1/items?per_page=&page=&folder_id=&include=', headers=headers).json()
print("Folder: " + response['data'][0]['name'])
next_url = response['meta']['next_page_url']
response = requests.get(next_url, headers=headers).json()
print(response)
print("Folder: " + response['data'][0]['name'])
next_url = response['meta']['next_page_url']
response = requests.get('https://api.sortly.co/api/v1/items/17481596?include=custom_attributes%2Cphotos', headers=headers).json()
photo_url = response['data']['photos'][0]['url']
print(response['data']['custom_attribute_values'])
response = requests.get(photo_url)
img = Image.open(io.BytesIO(response.content))
print("here's an item with a photo")
img.show()
# Posting test object
url = 'https://api.sortly.co/api/v1/items'

data = {'name' : "test object", 'type' : 'item', 'tags' : [{'name' : 'test'}]}
# print(url, data, headers)
response = requests.post(url, json=data, headers=headers)

Folder: Food
{'meta': {'page': 2, 'next_page_url': 'https://api.sortly.co/api/v1/items?folder_id=&include=&page=3&per_page=', 'total_pages': 6, 'total_count': 6}, 'data': [{'id': 17356162, 'name': 'Durable Things', 'price': None, 'quantity': None, 'min_quantity': None, 'notes': None, 'parent_id': None, 'sid': 'S3YNFR0002', 'label_url': None, 'label_url_type': None, 'label_url_extra': None, 'label_url_extra_type': None, 'tag_names': [], 'type': 'folder', 'created_at': '2020-09-30T13:32:45.181Z', 'updated_at': '2020-10-04T11:42:09.498Z'}]}
Folder: Durable Things
[{'value': '2020-09-05T04:00:00.000Z', 'custom_attribute_id': 170931, 'custom_attribute_name': 'Purchase Date'}, {'value': None, 'custom_attribute_id': 171873, 'custom_attribute_name': 'Product URL'}]
here's an item with a photo


### To Do
Various things to do:
- IMAP integration to get mail
- List of source and formats
- Trigger when there is a new email

In [ ]:
# gmail/quickstart/quickstart.py

from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)

    # Call the Gmail API
    results = service.users().labels().list(userId='me').execute()
    labels = results.get('labels', [])

    if not labels:
        print('No labels found.')
    else:
        print('Labels:')
        for label in labels:
            print(label['name'])

if __name__ == '__main__':
    main()

In [118]:
# Unit Tests
import unittest

class TestNotebook(unittest.TestCase):
    
    def test_get_file_date(self):
        # check date formatr
        inventory, date = get_file(data_file)
        self.assertTrue(re.match(r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.000Z', date))

unittest.main(argv=[''], verbosity=2, exit=False)

test_get_file_date (__main__.TestNotebook) ... 

opening fileemail_data/f2u-oneitem.txt


ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
